<a href="https://colab.research.google.com/github/TEKERGUL/hub_rout/blob/main/kgmtsp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install gurobipy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import json
import gurobipy as gp
from gurobipy import GRB
import numpy as np
import networkx as nx
try:
  sehirler_json = json.load(open('cities1.json'))
except:
  import urllib.request
  url = 'https://raw.githubusercontent.com/TEKERGUL/hub_rout/main/cities1.json'
  data = urllib.request.urlopen(url).read()
  sehirler_json = json.loads(data)

plakalar = []
sehirler = []
coordinates = {}
for state in sehirler_json:
      sehir = sehirler_json[state]['name']
      sehirler.append(sehir)
      coordinates[sehir] = (float(sehirler_json[state]['lat']), float(sehirler_json[state]['long']))

for state in sehirler_json:
      plaka = sehirler_json[state]['id']
      plakalar.append(plaka)
      coordinates[plaka] = (float(sehirler_json[state]['lat']), float(sehirler_json[state]['long']))

kgmc = 'https://raw.githubusercontent.com/TEKERGUL/hub_rout/main/kgm.csv'

arr = np.genfromtxt(kgmc, delimiter= ";")

arr[0,0] = 0
n = 81                                  # number of cities
points=[]
n=arr.shape[0]
G = nx.complete_graph(n, nx.DiGraph())
dist = {(i,j) : arr[i][j] for i in range(n) for j in range(n) if i != j}


{(0, 1): 338.0,
 (0, 2): 579.0,
 (0, 3): 978.0,
 (0, 4): 604.0,
 (0, 5): 492.0,
 (0, 6): 548.0,
 (0, 7): 1005.0,
 (0, 8): 881.0,
 (0, 9): 903.0,
 (0, 10): 786.0,
 (0, 11): 633.0,
 (0, 12): 746.0,
 (0, 13): 689.0,
 (0, 14): 649.0,
 (0, 15): 857.0,
 (0, 16): 1097.0,
 (0, 17): 591.0,
 (0, 18): 582.0,
 (0, 19): 762.0,
 (0, 20): 538.0,
 (0, 21): 1183.0,
 (0, 22): 496.0,
 (0, 23): 677.0,
 (0, 24): 809.0,
 (0, 25): 691.0,
 (0, 26): 216.0,
 (0, 27): 720.0,
 (0, 28): 780.0,
 (0, 29): 910.0,
 (0, 30): 196.0,
 (0, 31): 619.0,
 (0, 32): 69.0,
 (0, 33): 951.0,
 (0, 34): 908.0,
 (0, 35): 1014.0,
 (0, 36): 696.0,
 (0, 37): 335.0,
 (0, 38): 1160.0,
 (0, 39): 377.0,
 (0, 40): 841.0,
 (0, 41): 358.0,
 (0, 42): 676.0,
 (0, 43): 397.0,
 (0, 44): 887.0,
 (0, 45): 198.0,
 (0, 46): 549.0,
 (0, 47): 858.0,
 (0, 48): 743.0,
 (0, 49): 288.0,
 (0, 50): 210.0,
 (0, 51): 708.0,
 (0, 52): 918.0,
 (0, 53): 803.0,
 (0, 54): 754.0,
 (0, 55): 721.0,
 (0, 56): 868.0,
 (0, 57): 428.0,
 (0, 58): 1082.0,
 (0, 59): 493.0,
 

In [ ]:
e = gp.Env(empty=True)
e.setParam('WLSACCESSID', 'f5785d60-bc66-47fe-8503-f14d0784d56f')
e.setParam('WLSSECRET', 'e5cece14-d95a-47a2-b77d-b2803f2b1ba3')
e.setParam('LICENSEID', 894677)
e.start()
m = gp.Model(env=e)

Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 894677
Academic license - for non-commercial use only - registered to berketekergul@eskisehir.edu.tr


marshmallow for json capsules


In [ ]:
m = gp.Model(env=e)
x = m.addVars(G.edges, vtype=GRB.BINARY)
u = m.addVars( G.nodes )


depot = 5

m.setObjective( gp.quicksum( arr[i,j] * x[i,j] for i,j in G.edges ), GRB.MINIMIZE )

# Enter each city once
m.addConstrs( gp.quicksum( x[i,j] for i in G.predecessors(j) ) == 1 for j in G.nodes )

# Leave each city once
m.addConstrs( gp.quicksum( x[i,j] for j in G.successors(i) ) == 1 for i in G.nodes )
#depot
m.addConstr( gp.quicksum( x[depot,j]for j  in G.successors(depot)) == 1)
m.addConstr( gp.quicksum( x[i,depot]for i  in G.predecessors(depot)) == 1)

#MTZ
#c = m.addConstrs( u[i] - u[j] + n * x[i,j] <= n - 1 for i,j in G.edges if j != 0 )
m.addConstrs( u[i] - u[j] + (n-1) * x[i,j] + (n-3) * x[j,i] <= n-2 for i,j in G.edges if j != 0 )

m.optimize()

Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (linux64)

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Academic license - for non-commercial use only - registered to berketekergul@eskisehir.edu.tr
Optimize a model with 6564 rows, 6561 columns and 38720 nonzeros
Model fingerprint: 0x896d4ccf
Variable types: 81 continuous, 6480 integer (6480 binary)
Coefficient statistics:
  Matrix range     [1e+00, 8e+01]
  Objective range  [3e+01, 2e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 8e+01]
Presolve removed 2 rows and 1 columns
Presolve time: 0.09s
Presolved: 6562 rows, 6560 columns, 38480 nonzeros
Variable types: 80 continuous, 6480 integer (6480 binary)

Root relaxation: objective 9.368500e+03, 268 iterations, 0.02 seconds (0.01 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    B

In [ ]:
import copy
import sys

In [ ]:
tour_edges = [ e for e in G.edges if x[e].x > 0.5 ]
total=0
for i in range(81):
  total += dist[tour_edges[i]]

sortedNodes = [depot]
def findNext(edges, current):
  for pair in edges:
    if pair[0] == current: return pair[1]

for i in range(len(tour_edges)):
  sortedNodes.append(findNext(tour_edges, sortedNodes[len(sortedNodes) - 1]))

print("Seçilen Rota: ", sortedNodes)

print("Mesafe: ", total, "km")

Seçilen Rota:  [5, 25, 42, 2, 63, 31, 14, 6, 19, 47, 8, 34, 44, 9, 16, 21, 38, 58, 33, 40, 76, 15, 10, 53, 80, 13, 66, 73, 77, 36, 56, 54, 51, 27, 60, 52, 7, 74, 35, 75, 3, 24, 68, 28, 23, 61, 43, 22, 11, 48, 12, 64, 29, 72, 55, 71, 20, 46, 62, 1, 45, 26, 78, 30, 79, 0, 32, 69, 41, 67, 50, 49, 37, 57, 59, 4, 18, 65, 39, 70, 17, 5]
Mesafe:  9883.0 km


In [ ]:
yol= np.array(sortedNodes)
yol1=[]
for i in yol:
    yol1.append(sehirler[i])

In [ ]:
import folium

map1 = folium.Map(location=[39,35.5], zoom_start = 7)
points1 = []

for city in yol1:
  points1.append(coordinates[city])
points1.append(points1[0])


folium.Marker(points1[0], tooltip='marker').add_to(map1)
folium.PolyLine(points1).add_to(map1)
map1


In [ ]:
points

[]